1-VERİ HAZIRLAMA AŞAMASI


Gerekli kütüphaneleri çağıralım 

In [ ]:
#Genel komutlar

import numpy as np

import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
from sklearn.preprocessing import MinMaxScaler
import os
import math
#RMSE ile tahmin hatalarımı belirlemek için sqrt çağırdım.(evaluate forecast)
from math import sqrt
from sklearn.metrics import mean_squared_error
#Verisetini ayıklamak için çağırdım
from numpy import split
from numpy import array
#TimeSerieslerde kullanılan kütüphaneler
from datetime import timedelta
#Tensorflow kütüphaneleri

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence


Öğrenme setini çağırdım.Verisetimi google drive'a kaydedip çağırdım.



In [ ]:
path = "../input/solar-radiation-dataset/solar_angles_dataset.csv"
df = pd.read_csv(path)
df.head()

Verisetini oluşturuken oluşan boş sütunu sildim

In [ ]:
df.info()

In [ ]:
df=df.drop(['Julian day',"Top. azimuth angle (eastward from N)","Topocentric zenith angle"], axis = 1) 
df=df

-Zaman sütunlarını birleştirip datetime'a çevirdim, sonra diğer sütunları sildim.

-Datetime sütununu index yaptım .

In [ ]:
cols = ["Date (M/D/YYYY)","Time (H:MM:SS)"]
df["date_time"] = df[cols].apply(lambda row: "".join(row.values.astype(str)), axis=1)

In [ ]:
df['date_time'] = pd.to_datetime(df['date_time'], format='%m/%d/%Y%H:%M:%S')

In [ ]:
df=df.drop(["Date (M/D/YYYY)","Time (H:MM:SS)" ,"Unnamed: 0"], axis = 1)

In [ ]:
# Split into training, validation and test datasets.
# Since it's timeseries we should do it by date.
test_cutoff_date = df['date_time'].max() - timedelta(days=1)
val_cutoff_date = test_cutoff_date - timedelta(days=14)

df_test = df[df['date_time'] > test_cutoff_date]
df_val = df[(df['date_time'] > val_cutoff_date) & (df['date_time'] <= test_cutoff_date)]
df_train = df[df['date_time'] <= val_cutoff_date]

#check out the datasets
print('Test dates: {} to {}'.format(df_test['date_time'].min(), df_test['date_time'].max()))
print('Validation dates: {} to {}'.format(df_val['date_time'].min(), df_val['date_time'].max()))
print('Train dates: {} to {}'.format(df_train['date_time'].min(), df_train['date_time'].max()))

In [ ]:
df_test=df_test.set_index('date_time') #Columnu index yapmak için
df_val=df_val.set_index('date_time') #Columnu index yapmak için
df_train=df_train.set_index('date_time') #Columnu index yapmak için

In [ ]:
df_train.plot(figsize=(16,8))

In [ ]:
len(df_train)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df_train)

In [ ]:
scaled_train=scaler.transform(df_train)
scaled_test =scaler.transform(df_test)
scaled_val=scaler.transform(df_val)

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
n_input =4
n_features =1
train_generator = TimeseriesGenerator(scaled_train,scaled_train,length=n_input,batch_size = 1)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
model =Sequential()
model.add(LSTM(64,activation ="relu",input_shape=(n_input,n_features)))
model.add(Dense(1))
model.compile(optimizer ="adam",loss="mse")
model.summary()


In [ ]:
model.fit_generator(train_generator,epochs=10)

In [ ]:
model.history.history.keys()  

In [ ]:
myloss = model.history.history["loss"]
plt.plot(range(len(myloss)),myloss)

Forecast Visualisation

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(df_test)):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]
    
    # store prediction
    test_predictions.append(current_pred) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
true_predictions = scaler.inverse_transform(test_predictions)

In [ ]:
true_predictions

In [ ]:
df_test['Predictions'] = true_predictions

In [ ]:
df_test.plot(figsize=(20,8))